# SHAPASH

In [4]:
pip install --upgrade pandas

     ---------------------------------------- 10.8/10.8 MB 4.3 MB/s eta 0:00:00
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3



ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\leoma\\anaconda3\\Lib\\site-packages\\~andas\\_libs\\algos.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
pip install lightgbm

In [6]:

import pandas as pd
from category_encoders import OrdinalEncoder
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from shapash import SmartExplainer


C:\Users\leoma\anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\leoma\anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
C:\Users\leoma\anaconda3\lib\site-packages\dask\dataframe\utils.py:366: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


ModuleNotFoundError: No module named 'shapash'

In [ ]:
y_df=data['depart'].to_frame()
X_df=data[data.columns.difference(['depart'])]

In [ ]:
data.shape

data.head()

In [ ]:
#from category_encoders import OrdinalEncoder

categorical_features = [col for col in X_df.columns if X_df[col].dtype == 'object']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True).fit(X_df)

X_df=encoder.transform(X_df)

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_df, y_df, train_size=0.75, random_state=1)

In [ ]:
regressor = LGBMRegressor(n_estimators=100).fit(Xtrain,ytrain)

In [ ]:
xpl = SmartExplainer(
    model=regressor,
)

In [ ]:
xpl.compile(x=Xtest,
            y_target=ytest # Optional: allows to display True Values vs Predicted Values
           )

In [ ]:
#affiche le dashboard sur navigateur

app = xpl.run_app(title_story='Départ Employés', port=8020)

In [ ]:
##cloture dashboard

app.kill()

In [ ]:
summary_df= xpl.to_pandas(
    max_contrib=3, # Number Max of features to show in summary
    #threshold=5000,
)

summary_df.head()

In [ ]:
#compression format binaire doc pickle
#xpl.save('./xpl.pkl')